# Part 2 : Automatic correction

In [1]:
import nltk
nltk.download('words')
from nltk.corpus import words
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package words to /Users/jason13nn/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jason13nn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
class AutoCorrectText: 
        
    def __init__(self):
        pass
#----------------------------------------------------------------- 
    #Find the edit distance between two words
    def editDistance(self, str1, str2, deleteCost=1, defaultSubCost=2, subCostList={}): 
    
        #Convert into lowercase
    
        str1 = str(np.char.lower(str1))
        str2 = str(np.char.lower(str2))
    
        #Creating array (initialized with 0 of dimension of size of both strings)
    
        matrix = np.zeros((len(str1)+1,len(str2)+1))
    
    
        # Cross relation loop through each character of each string with each other and
        # fill the respective index of matrxi (row,column)
    
        for i in range(len(str1)+1): 
            for j in range(len(str2)+1): 
            
                #Only string 1 is empty
                if i == 0:  
                    matrix[i][j] = j  
                #Only string 2 is empty
                elif j == 0: 
                    matrix[i][j] = i
                #Both strings are empty
                elif i ==0 and j ==0:
                    matrix[i][j] = np.zeros((i,j))
                else: 
                    # Default cost of insertion/deletion=1 
                    insercost = matrix[i][j-1] + deleteCost
                    deletecot = matrix[i-1][j] + deleteCost
                    if str1[i-1] != str2[j-1]:
                        if subCostList == {}:
                            #Default cost of substitution=2
                            subcost = matrix[i-1][j-1] + defaultSubCost
                        else:
                            if tuple(sorted([str1[i-1],str2[j-1]])) in subCostList:
                                subcost = matrix[i-1][j-1] + subCostList[tuple(sorted([str1[i-1],str2[j-1]]))]
                            else:
                                subcost = matrix[i-1][j-1] + defaultSubCost
                    else:
                        subcost = matrix[i-1][j-1] + 0
                
                    matrix[i][j] = min(insercost,  
                                       deletecot,        
                                       subcost)     
                                         
    
        return matrix[len(str1)][len(str2)]  # Returning the final
#-----------------------------------------------------------------   
    #Autocorrect the text
    def Correct(self, text):
        #Create a dictionary for autocorrect
        l = words.words()
        #Convert to lower case
        l = [w.lower() for w in l] 
        word = nltk.word_tokenize(text)
        word = [v.lower() for v in word]
        punctuation = [',', '.', '?', '!', ':', ';', '-']
        
        pos = []
        w1 = []
        w2 = []
        
        for w in word:
            if w not in l and w not in punctuation:
                #1. The position of the words
                ind = word.index(w)
                pos.append(ind)
                #2. The original word
                w1.append(w)
                #3. The corrected words
                
                scores = {}
                for d in l:
                    scores[d] = self.editDistance(w, d)
                    min_dis = min(scores, key = scores.get)
                w2.append(min_dis)

                #Merge lists into a list of tuples
            merged_list = tuple(zip(pos, w1, w2)) 
        
        return merged_list